In [33]:
import requests

# results 2016
url = "https://en.wikipedia.org/wiki/2016_United_States_presidential_election#Results_by_state"
response = requests.get(url)
response

<Response [200]>

In [35]:
import pandas as pd
from bs4 import BeautifulSoup
soup = BeautifulSoup(response.content)


In [36]:
table1 = soup.find('div', style='overflow:auto')


In [39]:
# filtering all td tags to get state & percentages as list 

table_list1 = []
for td in table1.find_all('td'):
        table_list1.append(td.text.strip())  

In [41]:
# indexing the elements

for index, element in enumerate(table_list1):
    print(index, element)

0 Alabama
1 729,547
2 34.36%
3 –
4 1,318,255
5 62.08%
6 9
7 44,467
8 2.09%
9 –
10 9,391
11 0.44%
12 –
13 –
14 –
15 –
16 21,712
17 1.02%
18 –
19 588,708
20 27.73%
21 2,123,372
22 [400]
23 Alaska
24 116,454
25 36.55%
26 –
27 163,387
28 51.28%
29 3
30 18,725
31 5.88%
32 –
33 5,735
34 1.80%
35 –
36 –
37 –
38 –
39 14,307
40 4.49%
41 –
42 46,933
43 14.73%
44 318,608
45 [401]
46 Arizona
47 1,161,167
48 44.58%
49 –
50 1,252,401
51 48.08%
52 11
53 106,327
54 4.08%
55 –
56 34,345
57 1.32%
58 –
59 17,449
60 0.67%
61 –
62 32,968
63 1.27%
64 –
65 91,234
66 3.50%
67 2,604,657
68 [402]
69 Arkansas
70 380,494
71 33.65%
72 –
73 684,872
74 60.57%
75 6
76 29,949
77 2.64%
78 –
79 9,473
80 0.84%
81 –
82 13,176
83 1.17%
84 –
85 12,712
86 1.12%
87 –
88 304,378
89 26.92%
90 1,130,676
91 [403]
92 California
93 8,753,788
94 61.73%
95 55
96 4,483,810
97 31.62%
98 –
99 478,500
100 3.37%
101 –
102 278,657
103 1.96%
104 –
105 39,596
106 0.28%
107 –
108 147,244
109 1.04%
110 –
111 −4,269,978
112 −30.11%
113 14,181,5

In [43]:
# searching for elements with letters, group them into sublists with state name as first index (listname:states_data)
import re       

states_data = []
current_state = []

for element in table_list1:
    if re.search(r'[a-zA-Z]', element):  
        if current_state:
            states_data.append(current_state)
        current_state = [element]  
    else:
        current_state.append(element)
if current_state:
    states_data.append(current_state)


In [45]:
# filtering the list, create dicts with: states (index[0]) hillary votes (index[2]) donald votes (index[5])
result_votes_2016 = {}

for entry in states_data:
        state = entry[0]
        hillary_votes = entry[2]
        trump_votes = entry[5]
        
        result_votes_2016[state] = {
        "Hillary Clinton": hillary_votes,
        "Donald Trump": trump_votes
    }

print(result_votes_2016)

{'Alabama': {'Hillary Clinton': '34.36%', 'Donald Trump': '62.08%'}, 'Alaska': {'Hillary Clinton': '36.55%', 'Donald Trump': '51.28%'}, 'Arizona': {'Hillary Clinton': '44.58%', 'Donald Trump': '48.08%'}, 'Arkansas': {'Hillary Clinton': '33.65%', 'Donald Trump': '60.57%'}, 'California': {'Hillary Clinton': '61.73%', 'Donald Trump': '31.62%'}, 'Colorado': {'Hillary Clinton': '48.16%', 'Donald Trump': '43.25%'}, 'Connecticut': {'Hillary Clinton': '54.57%', 'Donald Trump': '40.93%'}, 'Delaware': {'Hillary Clinton': '53.09%', 'Donald Trump': '41.72%'}, 'District of Columbia': {'Hillary Clinton': '90.86%', 'Donald Trump': '4.09%'}, 'Florida': {'Hillary Clinton': '47.82%', 'Donald Trump': '49.02%'}, 'Georgia': {'Hillary Clinton': '45.64%', 'Donald Trump': '50.77%'}, 'Hawaii': {'Hillary Clinton': '62.22%', 'Donald Trump': '30.03%'}, 'Idaho': {'Hillary Clinton': '27.49%', 'Donald Trump': '59.26%'}, 'Illinois': {'Hillary Clinton': '55.83%', 'Donald Trump': '38.76%'}, 'Indiana': {'Hillary Clinton

In [47]:
# dataframe: states as indexes, percentages are saved as strings!

df_results_2016 = pd.DataFrame.from_dict(result_votes_2016, orient='index')
df_results_2016.index.name = 'State'
df_results_2016.columns = ['Hillary Clinton', 'Donald Trump']

In [55]:
# scraping for total percentages 2016

values = soup.find_all('th', style="text-align:right")
total_values_2016 = []

for th in values:
    text = th.get_text(strip=True)  
    if "%" in text: 
        percent_values.append(text)
total_values_2016 = [value for value in percent_values if value != '%']
total_values_2016 = cleaned_percent_list[:2]

total_values_2016

['48.18%', '46.09%']

In [75]:
# adding total votes as last row 

df_results_2016 = pd.DataFrame.from_dict(result_votes_2016, orient='index')

df_results_2016.index.name = 'State'
df_results_2016.columns = ['Hillary Clinton', 'Donald Trump']
total_votes = ['48.18%', '46.09%']
df_results_2016.loc['National'] = total_values_2016

print(df_results_2016)  

                                                  Hillary Clinton Donald Trump
State                                                                         
Alabama                                                    34.36%       62.08%
Alaska                                                     36.55%       51.28%
Arizona                                                    44.58%       48.08%
Arkansas                                                   33.65%       60.57%
California                                                 61.73%       31.62%
Colorado                                                   48.16%       43.25%
Connecticut                                                54.57%       40.93%
Delaware                                                   53.09%       41.72%
District of Columbia                                       90.86%        4.09%
Florida                                                    47.82%       49.02%
Georgia                                             

In [59]:
df_results_2016.index.value_counts().sum()

57

In [61]:
# results 2020

url = "https://en.m.wikipedia.org/wiki/2020_United_States_presidential_election#Results"
response = requests.get(url)
response

<Response [200]>

In [65]:
soup2 = BeautifulSoup(response.content)
table2 = soup2.find('div', style='overflow:auto')

In [67]:
# all percentages & states

table_list2 = []
for td in table2.find_all('td'):
        table_list2.append(td.text.strip())

In [69]:
for index, element in enumerate(table_list2):
    print(index, element)

0 Alabama
1 849,624
2 36.57%
3 –
4 1,441,170
5 62.03%
6 9
7 25,176
8 1.08%
9 –
10 [o]
11 [o]
12 –
13 7,312
14 0.31%
15 –
16 −591,546
17 −25.46%
18 2.27%
19 2,323,282
20 Alaska
21 153,778
22 42.77%
23 –
24 189,951
25 52.83%
26 3
27 8,897
28 2.47%
29 –
30 [p]
31 [p]
32 –
33 6,904
34 1.92%
35 –
36 −36,173
37 −10.06%
38 4.67%
39 359,530
40 Arizona
41 1,672,143
42 49.36%
43 11
44 1,661,686
45 49.06%
46 –
47 51,465
48 1.52%
49 –
50 1,557
51 0.05%
52 –
53 475
54 0.01%
55 –
56 10,457
57 0.31%
58 3.81%
59 3,387,326
60 Arkansas
61 423,932
62 34.78%
63 –
64 760,647
65 62.40%
66 6
67 13,133
68 1.08%
69 –
70 2,980
71 0.24%
72 –
73 18,377
74 1.51%
75 –
76 −336,715
77 −27.62%
78 −0.70%
79 1,219,069
80 California
81 11,110,639
82 63.48%
83 55
84 6,006,518
85 34.32%
86 –
87 187,910
88 1.07%
89 –
90 81,032
91 0.46%
92 –
93 115,281
94 0.66%
95 –
96 5,104,121
97 29.16%
98 −0.95%
99 17,501,380
100 Colorado
101 1,804,352
102 55.40%
103 9
104 1,364,607
105 41.90%
106 –
107 52,460
108 1.61%
109 –
110 8,986
11

In [71]:
# creating sublists with state as first index
states_data2 = []
current_state2 = []

for element in table_list2:
    if re.search(r'[a-zA-Z]', element):  
        if current_state2:
            states_data2.append(current_state2)
        current_state2 = [element]  
    else:
        current_state2.append(element)
if current_state2:
    states_data2.append(current_state2)

In [73]:
#filtering state, b votes & d votes, save in dict

result_votes_2020 = {}

for entry in states_data2:
    if len(entry) > 5:
        state = entry[0]
        biden_votes = entry[2]
        trump_votes = entry[5]
        
    result_votes_2020[state] = {
        "Joseph Biden": biden_votes,
        "Donald Trump": trump_votes
    }
print(result_votes_2020)

{'Alabama': {'Joseph Biden': '36.57%', 'Donald Trump': '62.03%'}, '[o]': {'Joseph Biden': '3,947', 'Donald Trump': '−120,068'}, 'Alaska': {'Joseph Biden': '42.77%', 'Donald Trump': '52.83%'}, '[p]': {'Joseph Biden': '6,904', 'Donald Trump': '−36,173'}, 'Arizona': {'Joseph Biden': '49.36%', 'Donald Trump': '49.06%'}, 'Arkansas': {'Joseph Biden': '34.78%', 'Donald Trump': '62.40%'}, 'California': {'Joseph Biden': '63.48%', 'Donald Trump': '34.32%'}, 'Colorado': {'Joseph Biden': '55.40%', 'Donald Trump': '41.90%'}, 'Connecticut': {'Joseph Biden': '59.26%', 'Donald Trump': '39.19%'}, 'Delaware': {'Joseph Biden': '58.74%', 'Donald Trump': '39.77%'}, 'District of Columbia': {'Joseph Biden': '92.15%', 'Donald Trump': '5.40%'}, 'Florida': {'Joseph Biden': '47.86%', 'Donald Trump': '51.22%'}, 'Georgia': {'Joseph Biden': '49.47%', 'Donald Trump': '49.24%'}, 'Hawaii': {'Joseph Biden': '63.73%', 'Donald Trump': '34.27%'}, 'Idaho': {'Joseph Biden': '33.07%', 'Donald Trump': '63.84%'}, 'Illinois': {

In [77]:
# scraping for total percentages

values2 = soup2.find_all('th', style="text-align:right")
percent_values2 = []
for th in values2:
    text = th.get_text(strip=True)  
    if "%" in text: 
        percent_values2.append(text)
cleaned_percent_list2 = [value for value in percent_values2 if value != '%']
total_percentage_2020 = cleaned_percent_list2[:2]

total_percentage_2020

['51.31%', '46.85%']

In [79]:
df_results_2020 = pd.DataFrame.from_dict(result_votes_2020, orient='index')

df_results_2020.index.name = 'State'
df_results_2020.columns = ['Joseph Biden', 'Donald Trump']
df_results_2020 = df_results_2020.drop(['[o]', '[p]'], axis=0)

df_results_2020.loc['National'] = total_percentage_2020
print(df_results_2020)

                                                  Joseph Biden Donald Trump
State                                                                      
Alabama                                                 36.57%       62.03%
Alaska                                                  42.77%       52.83%
Arizona                                                 49.36%       49.06%
Arkansas                                                34.78%       62.40%
California                                              63.48%       34.32%
Colorado                                                55.40%       41.90%
Connecticut                                             59.26%       39.19%
Delaware                                                58.74%       39.77%
District of Columbia                                    92.15%        5.40%
Florida                                                 47.86%       51.22%
Georgia                                                 49.47%       49.24%
Hawaii      

In [81]:
df_results_2020.value_counts().sum()

57